## Session Overview

In this session, we will:

- Run Ollama locally and explore its capabilities.
- Install the `llama3:2` (latest) model.
- Compare the outputs of three language models: **GPT**, **Gemini**, and **Llama**.
- Use GPT as a judge to evaluate the outputs (LLM-as-judge approach).
- Explore generating structured outputs using **Pydantic**.

## Prerequisites

- **Install Ollama**: Follow the instructions at [Ollama's official website](https://ollama.com/download) to install Ollama on your machine.
- **Download the Llama 3 model**: Run the following command in your terminal to download and start the Llama 3 model:
  
  ```
  ollama run llama3.2:1b
  ```

  This will ensure the model is available locally for comparison.
- **Gemini API Key (Optional)**: Sign-up on Gemini for a free account, create an API key and store it in your .env file as `GOOGLE_API_KEY`. See `.env.example` for reference.

In [ ]:
# import necessary libraries
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
from pydantic import BaseModel

In [ ]:
# Load env variables and initiate clients for OpenAI, Gemini and Ollama (llama)

# Gemini Base URL: https://generativelanguage.googleapis.com/v1beta/openai/
# Gemini Model: gemini-2.0-flash
# Gemini Models: https://ai.google.dev/gemini-api/docs/models

# Ollama Base URL: http://localhost:11434/v1
# Ollama Model: llama3.2

load_dotenv(override=True)
openai_client = OpenAI()
gemini_client = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
ollama_client = OpenAI(base_url="http://localhost:11434/v1")

In [ ]:
# Ask GPT to generate a nuanced question that can be asked to GPT, Gemini and Ollama to judge their capabilities

prompt = "Please come up with a challenging, nuanced question that can be asked to an LLM to evaluate its intelligence. Answer only with a question, no explanation"
messages = [{"role": "system", "content": prompt}]

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

challenging_question = response.choices[0].message.content
print(challenging_question)

In [ ]:
# Prepare messages for LLMs with question to answer

messages = [{"role": "user", "content": challenging_question}]

In [ ]:
# Ask GPT to answer the question

gpt_response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

gpt_answer = gpt_response.choices[0].message.content
print(gpt_answer)

In [ ]:
# Ask Gemini to answer the question

gemini_response = gemini_client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=messages
)

gemini_answer = gemini_response.choices[0].message.content
print(gemini_answer)

In [ ]:
# Ask Ollama to answer the question

ollama_response = ollama_client.chat.completions.create(
    model="llama3.2",
    messages=messages
)

ollama_answer = ollama_response.choices[0].message.content
print(ollama_answer)

In [ ]:
# Ask GPT 4.1 to judge the answers based on clarity and strength of the argument.
# GPT should respond with a json object.

prompt = f"""You are judging a competition between GPT, Gemini and Llama.
Each model has been give the following question:
{challenging_question}

Your job is to evaluate each response based on clarity and strength of the argument, ranking them best to worst and give a one line explanation for your ranking.

Response with json and only with json in the following format:
{'{response: [{rank: number, explanation: string, model: string}]}'}

GPT:
{gpt_answer}

Gemini:
{gemini_answer}

Llama:
{ollama_answer}

Now respond with the json object only. Do not include markdown formatting or code blocks.
"""

prompt

In [ ]:
# Send judge prompt to GPT 4.1

judge_response = openai_client.chat.completions.create(
    model="gpt-4.1-2025-04-14",
    messages=[{"role": "user", "content": prompt}],
)

judge_output = judge_response.choices[0].message.content
print(judge_output)

In [ ]:
# Parse judge output to json

judge_output_json = json.loads(judge_output)
print(judge_output_json)

### Is there a better way to obtain structured output?

Yes, you can use **Pydantic** to define a schema for the expected output and validate the response, ensuring it adheres to the desired structure.

In [ ]:
# define pydantic schema for judge's structured output

class Judgement(BaseModel):
    rank: int
    explanation: str
    model: str

class JudgeOutput(BaseModel):
    response: list[Judgement]

In [ ]:
# Rewrite the judge prompt. This time, without the json object.

prompt = f"""You are judging a competition between GPT, Gemini and Llama.
Each model has been give the following question:
{challenging_question}

Your job is to evaluate each response based on clarity and strength of the argument, ranking them best to worst and give a one line explanation for your ranking.

GPT:
{gpt_answer}

Gemini:
{gemini_answer}

Llama:
{ollama_answer}
"""

prompt

In [ ]:
# Send judge prompt to GPT 4.1 again
# Instead of Chat Completions create, use parse this time

judge_response = openai_client.chat.completions.parse(
    model="gpt-4.1-2025-04-14",
    messages=[{"role": "user", "content": prompt}],
    response_format=JudgeOutput
)

judge_output = judge_response.choices[0].message.content

judge_output_json = json.loads(judge_output)
print(json.dumps(judge_output_json, indent=4))